In [1]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [2]:
# Importing self-made libaries
import standards as s
import life_cycle_assessment as lc
import LCA_plots as lp
import reload_lib as rl
import non_bio_co2 as nbc


lib = [s, lc, lp, nbc]

In [3]:
from copy import deepcopy as dc

In [4]:
import bw2io as bi
import bw2data as bd
import pandas as pd

In [5]:
bd.projects.set_current("single use vs multi use")


In [6]:
if any("biosphere" in db for db in bd.databases):
    print('Biosphere is already present in the project.')
else:
    bi.bw2setup()

Biosphere is already present in the project.


In [7]:
if 'ev391cutoff' in bd.databases and 'ev391consq' in bd.databases and 'ev391apos' in bd.databases:
    print('Ecoinvent 3.9.1 is already present in the project.')
else:
    # APOS
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets", db_name='ev391apos') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # Consequential
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets", db_name='ev391consq') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # cut-off
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Downloads\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets", db_name='ev391cutoff') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

Ecoinvent 3.9.1 is already present in the project.


In [8]:
bd.databases

Databases dictionary with 10 object(s):
	biosphere3
	case1_apos
	case1_consq
	case1_cut_off
	case2_apos
	case2_consq
	case2_cut_off
	ev391apos
	ev391consq
	ev391cutoff

In [92]:
system_path = [r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case1.xlsx', 
              r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case2.xlsx']

for path in system_path:    
    db_path = path 
    sheet_name = ['ev391apos', 'ev391consq', 'ev391cutoff'] # Replace with your actual sheet name

    for db in sheet_name:
        # print(idx+1, db)
        # Read the Excel file
        data = pd.read_excel(db_path, sheet_name=db)
        # Save the data to a temporary file that can be used by ExcelImporter
        temp_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\temp.xlsx'
        data.to_excel(temp_path, index=False)

        if data.columns[1] not in bd.databases:

            # Use the temporary file with ExcelImporter
            imp = bi.ExcelImporter(temp_path)  # the path to your inventory excel file
            imp.apply_strategies()

            # # List of databases to match

            # Loop through each database and match
            print(f"Matching database: {db}")
            imp.match_database(db, fields=('name', 'unit', 'location', 'reference product'))
            print(f"Unlinked items after matching {db}: {list(imp.unlinked)}")

            # Match without specifying a database
            imp.match_database(fields=('name', 'unit', 'location'))

            # Generate statistics and write results
            imp.statistics()
            imp.write_excel(only_unlinked=True)
            unlinked_items = list(imp.unlinked)
            imp.write_database()

            # Print unlinked items if needed
            print(unlinked_items)
            print(f'{data.columns[1]} is loaded into the database')
        else:
            print(f'{data.columns[1]} already exist')

case1_apos already exist
case1_consq already exist
case1_cut_off already exist
case2_apos already exist
case2_consq already exist
case2_cut_off already exist


In [14]:
db = bd.Database('case1_consq')
flow = []
if 'case1' in str(db):
    for act in db:
        temp = act['name']
        # print(flow)
        if ('H2' in temp  or 'H4' in temp) and ('SU' in temp or 'REC' in temp) and temp not in flow:
           flow.append(temp) 
        elif 'alubox' in temp and '+' in temp and 'eol' not in temp.lower():
            flow.append(temp) 
    flow.sort()
elif 'case2' in str(db):
    for act in db:
        temp = act['name']
        if temp == 'SUD' or temp == 'MUD':
            flow.append(temp)
    flow.sort()
    flow.reverse()

flow

['H200 REC',
 'H200 SU',
 'H400 REC',
 'H400 SU',
 'alubox (large + dishwasher)',
 'alubox (large + wipe)',
 'alubox (small + dishwasher)',
 'alubox (small + wipe)']

In [15]:
flows = flow

In [16]:
procces_keys = {key: None for key in flows}

size = len(flows)

for act in db:
        for proc in range(size):
            if act['name'] == flows[proc]:
                procces_keys[act['name']] = act['code']

procces_keys

{'H200 REC': 'b1178f93d030e078ebbae2af3fe76929',
 'H200 SU': '960b1b91a915ed45195f37b64e47d4c9',
 'H400 REC': '6c866d09cde0b3fa0daf28c657e28dc7',
 'H400 SU': '7b9d008c491d64e1b98d8809c2ec8331',
 'alubox (large + dishwasher)': '0a97ff39ffed072e23f98d2b27ca8c02',
 'alubox (large + wipe)': '65231d0406d7ca520fa7f8ab0ef57758',
 'alubox (small + dishwasher)': '7807e75f1bc0c9c573389f20179d9db1',
 'alubox (small + wipe)': 'b3dcf85f3c8b1ed940d2c6043a2f33b9'}

In [17]:
process = []
for key, item in procces_keys.items():
        try:
            process.append(db.get(item))
        except KeyError:
            print(f"Process with key '{item}' not found in the database '{db}'")
            process = None
process

['H200 REC' (unit, GLO, None),
 'H200 SU' (unit, GLO, None),
 'H400 REC' (unit, GLO, None),
 'H400 SU' (unit, GLO, None),
 'alubox (large + dishwasher)' (unit, GLO, None),
 'alubox (large + wipe)' (unit, GLO, None),
 'alubox (small + dishwasher)' (unit, GLO, None),
 'alubox (small + wipe)' (unit, GLO, None)]

In [18]:
product_details = {}
product_details_code = {}

# Obtaining the subsubprocess'
if process:
    for proc in process:
        product_details[proc['name']] = []
        product_details_code[proc['name']] = []

        for exc in proc.exchanges():
            if exc['type'] == 'technosphere' or exc['type'] == 'biosphere':
            # if 'Use' in exc.output['name'] and exc['type'] == 'biosphere':
                product_details[proc['name']].append({exc.input['name']: [exc['amount'], exc.input]})
                # if exc.input in eidb or exc.input in eidb_db or exc.input in eidb_cyl:
                #     product_details_code[proc['name']].append([exc.output, exc.output['name'], exc.output['code'], exc['amount']])
            # elif exc['type'] == 'technosphere':
            #     product_details[proc['name']].append({exc.input['name']: [exc['amount'], exc.input]})


In [19]:
FU = {key: {} for key in product_details.keys()}
# Creating the FU to calculate for
for key, item in product_details.items():
    for idx in item:
        for n, m in idx.items():
            FU[key].update({m[1]: m[0]})


In [20]:
FU

{'H200 REC': {'H200' (unit, GLO, None): 1,
  'autoclave' (unit, GLO, None): 0.08333333333333333,
  'waste polypropylene' (kilogram, CH, None): -0.0107,
  'waste polyethylene' (kilogram, CH, None): -0.0109,
  'market for electricity, high voltage' (kilowatt hour, DK, None): -0.022063716860625,
  'marginal heating grid' (megajoule, GLO, None): -0.39714690349125,
  'market for polypropylene, granulate' (kilogram, GLO, None): -0.0604,
  'polyethylene, high density, granulate, recycled to generic market for high density PE granulate' (kilogram, Europe without Switzerland, None): -0.00618,
  'polyethylene production, high density, granulate, recycled' (kilogram, Europe without Switzerland, None): 0.00727,
  'polyethylene terephthalate production, granulate, amorphous, recycled' (kilogram, Europe without Switzerland, None): 0.071,
  'treatment of waste paper, unsorted, sorting' (kilogram, RoW, None): -0.00316},
 'H200 SU': {'waste polypropylene' (kilogram, CH, None): 0.071,
  'waste polyethyl

In [21]:
unique_process_index = []
uniquie_process = []

for act, exc in FU.items():
    for proc in exc.keys():
        if str(proc) not in unique_process_index:
            unique_process_index.append(str(proc))
            uniquie_process.append(proc)
unique_process_index.sort()


In [22]:
FU1 = []
for upi in unique_process_index:
    for proc in uniquie_process:
        if upi == f'{proc}':
            FU1.append({proc: 1})


In [23]:
FU1

[{'H200' (unit, GLO, None): 1},
 {'H400' (unit, GLO, None): 1},
 {'alubox (large)' (unit, GLO, None): 1},
 {'alubox (small)' (unit, GLO, None): 1},
 {'aluminium scrap' (kilogram, RER, None): 1},
 {'autoclave' (unit, GLO, None): 1},
 {'marginal heating grid' (megajoule, GLO, None): 1},
 {'market for aluminium, cast alloy' (kilogram, GLO, None): 1},
 {'market for electricity, high voltage' (kilowatt hour, DK, None): 1},
 {'market for polypropylene, granulate' (kilogram, GLO, None): 1},
 {'mechanical disinfection' (unit, GLO, None): 1},
 {'polyethylene production, high density, granulate, recycled' (kilogram, Europe without Switzerland, None): 1},
 {'polyethylene terephthalate production, granulate, amorphous, recycled' (kilogram, Europe without Switzerland, None): 1},
 {'polyethylene, high density, granulate, recycled to generic market for high density PE granulate' (kilogram, Europe without Switzerland, None): 1},
 {'treatment of waste paper, unsorted, sorting' (kilogram, RoW, None): 1}

In [24]:
import bw2calc as bc

In [25]:
impact_categories = lc.lcia_method('recipe')
if type(impact_categories) == tuple:
    impact_categories = [ic for ic in impact_categories]
df_unique = pd.DataFrame(0, index=unique_process_index, columns=impact_categories, dtype=object)
# unique_process_results = {}
# calc_count = 1
# total_calculations = len(uniquie_process_dct)*len(impact_categories)
print(f'Calculating for {len(impact_categories)} methods and {len(FU1)} activities : Total calculations {len(impact_categories) * len(FU1)}' )
bd.calculation_setups['calc_setup'] = {'inv':FU1, 'ia': impact_categories}
mylca = bc.MultiLCA('calc_setup')
res = mylca.results
for col, arr in enumerate(res):
    for row, val in enumerate(arr):
        df_unique.iat[col, row] = val

df_unique

Recipe is selected
Calculating for 21 methods and 19 activities : Total calculations 399


,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, acidification: terrestrial, terrestrial acidification potential (TAP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, climate change, global warming potential (GWP1000))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: freshwater, freshwater ecotoxicity potential (FETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: marine, marine ecotoxicity potential (METP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: terrestrial, terrestrial ecotoxicity potential (TETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, energy resources: non-renewable, fossil, fossil fuel potential (FFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: freshwater, freshwater eutrophication potential (FEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: marine, marine eutrophication potential (MEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: carcinogenic, human toxicity potential (HTPc))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: non-carcinogenic, human toxicity potential (HTPnc))",...,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, land use, agricultural land occupation (LOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, material resources: metals/minerals, surplus ore potential (SOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ozone depletion, ozone depletion potential (ODPinfinite))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, particulate matter formation, particulate matter formation potential (PMFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: human health, photochemical oxidant formation potential: humans (HOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: terrestrial ecosystems, photochemical oxidant formation potential: ecosystems (EOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, water use, water consumption potential (WCP))","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: natural resources, natural resources)"
"'H200' (unit, GLO, None)",0.000555,0.218333,0.005877,0.007822,0.754893,0.143219,0.00005,0.000006,0.006116,0.123293,...,0.010123,0.000261,0.0,0.000294,0.000488,0.000521,0.001803,0.0,0.0,0.056194
"'H400' (unit, GLO, None)",0.001589,0.625175,0.016824,0.022396,2.158866,0.409834,0.000143,0.000016,0.01754,0.35349,...,0.02982,0.000749,0.0,0.000843,0.001397,0.001493,0.00519,0.0,0.000001,0.160763
"'alubox (large)' (unit, GLO, None)",0.000066,0.02425,0.001516,0.001993,0.108779,0.007827,0.000013,0.000002,0.001238,0.031457,...,0.002997,0.000064,0.0,0.000036,0.000042,0.000045,0.000642,0.0,0.0,0.002402
"'alubox (small)' (unit, GLO, None)",0.000039,0.01426,0.0009,0.001183,0.064124,0.004563,0.000008,0.000001,0.000734,0.018636,...,0.001792,0.000038,0.0,0.000021,0.000025,0.000027,0.000379,0.0,0.0,0.001392
"'aluminium scrap' (kilogram, RER, None)",0.170371,39.908048,0.802357,1.123208,-26.30871,8.557749,0.013017,0.00096,7.929035,28.635392,...,0.421665,0.524474,0.000007,0.077939,0.10311,0.105561,0.230817,0.0,0.000119,1.50898
"'autoclave' (unit, GLO, None)",0.00696,0.61558,0.269075,0.318025,24.638059,0.159362,-0.00022,0.001597,0.032595,1.379421,...,2.123279,0.003916,-0.000002,0.002887,0.00496,0.005064,0.006986,0.0,0.000003,0.027324
"'marginal heating grid' (megajoule, GLO, None)",0.000072,0.043407,0.001953,0.002503,0.196412,0.013238,0.000008,0.000001,0.000741,0.029447,...,0.01438,0.000047,0.0,0.000044,0.000096,0.000102,0.000128,0.0,0.0,0.004517
"'market for aluminium, cast alloy' (kilogram, GLO, None)",0.089817,20.591952,0.49193,0.68886,17.469344,4.469172,0.006761,0.000488,4.028008,17.215672,...,0.198174,0.252293,0.000004,0.040567,0.053196,0.054508,0.117071,0.0,0.000062,0.805702
"'market for ele

In [35]:
for col, impact in enumerate(impact_categories):
            for proc, fu in FU.items():
                for key, item in fu.items():
                    print(key,item)
                


'H200' (unit, GLO, None) 1
'autoclave' (unit, GLO, None) 0.08333333333333333
'treatment of waste polypropylene, municipal incineration with fly ash extraction' (kilogram, CH, None) 0.0107
'treatment of waste polyethylene, municipal incineration with fly ash extraction' (kilogram, CH, None) 0.0109
'market for electricity, high voltage' (kilowatt hour, DK, None) -0.14
'mixed heating grid' (megajoule, GLO, None) -0.49
'market for polypropylene, granulate' (kilogram, GLO, None) -0.0604
'polyethylene, high density, granulate, recycled to generic market for high density PE granulate' (kilogram, Europe without Switzerland, None) -0.00618
'polyethylene production, high density, granulate, recycled' (kilogram, Europe without Switzerland, None) 0.00727
'polyethylene terephthalate production, granulate, amorphous, recycled' (kilogram, Europe without Switzerland, None) 0.071
'treatment of waste paper, unsorted, sorting' (kilogram, RoW, None) 0.00316
'H200' (unit, GLO, None) 1
'autoclave' (unit, GL

In [26]:
functional_unit = FU
df_res= pd.DataFrame(0, index=flows, columns=impact_categories, dtype=object)

for col in impact_categories:
    for i, row in df_res.iterrows():
        row[col] = []

for col, impact in enumerate(impact_categories):
            for proc, fu in functional_unit.items():
                # print(proc)
                for key, item in fu.items():
                    exc = str(key)
                    val = float(item)
                    factor = df_unique.at[exc, impact]
                    impact_value = val * factor
                    df_res.at[proc, impact].append([exc, impact_value])
df_res        


,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, acidification: terrestrial, terrestrial acidification potential (TAP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, climate change, global warming potential (GWP1000))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: freshwater, freshwater ecotoxicity potential (FETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: marine, marine ecotoxicity potential (METP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: terrestrial, terrestrial ecotoxicity potential (TETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, energy resources: non-renewable, fossil, fossil fuel potential (FFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: freshwater, freshwater eutrophication potential (FEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: marine, marine eutrophication potential (MEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: carcinogenic, human toxicity potential (HTPc))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: non-carcinogenic, human toxicity potential (HTPnc))",...,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, land use, agricultural land occupation (LOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, material resources: metals/minerals, surplus ore potential (SOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ozone depletion, ozone depletion potential (ODPinfinite))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, particulate matter formation, particulate matter formation potential (PMFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: human health, photochemical oxidant formation potential: humans (HOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: terrestrial ecosystems, photochemical oxidant formation potential: ecosystems (EOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, water use, water consumption potential (WCP))","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: natural resources, natural resources)"
H200 REC,"[['H200' (unit, GLO, None), 0.0005546509604626...","[['H200' (unit, GLO, None), 0.2183325574785768...","[['H200' (unit, GLO, None), 0.0058766932851411...","[['H200' (unit, GLO, None), 0.0078223413254126...","[['H200' (unit, GLO, None), 0.7548931414728998...","[['H200' (unit, GLO, None), 0.1432192406002728...","[['H200' (unit, GLO, None), 4.9719562522939887...","[['H200' (unit, GLO, None), 5.551816891789363e...","[['H200' (unit, GLO, None), 0.0061164262559475...","[['H200' (unit, GLO, None), 0.1232933209096517...",...,"[['H200' (unit, GLO, None), 0.0101233208788197...","[['H200' (unit, GLO, None), 0.0002612648251432...","[['H200' (unit, GLO, None), 3.107943666280824e...","[['H200' (unit, GLO, None), 0.0002937187535435...","[['H200' (unit, GLO, None), 0.0004875848615805...","[['H200' (unit, GLO, None), 0.0005212374161921...","[['H200' (unit, GLO, None), 0.0018029811551682...","[['H200' (unit, GLO, None), 9.329891671577168e...","[['H200' (unit, GLO, None), 4.401419971313193e...","[['H200' (unit, GLO, None), 0.0561938466120716..."
H200 SU,"[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...",...,"[['waste polypropylene' (kilogram, CH, None), ...","[['waste polypropylene' (kilogram, CH, None), ...","[['waste poly

In [57]:
rl.reload_lib(lib)
temp = lp.category_organization('case1')
temp

for lst in temp.values():
    for item in lst:
        print(item)


Raw mat. + prod.
Disinfection
Autoclave
Recycling
Incineration
Avoided energy prod.
Avoided mat. prod.
Total
